In [1]:
from lava.lib.dl.netx import hdf5
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader,Subset
from matplotlib import animation
import matplotlib.pyplot as plt

import numpy as np
 
from lava.proc import io
from lava.magma.core import run_configs as rcfg
from lava.magma.core import run_conditions as rcnd
import lava.lib.dl.slayer as slayer


ModuleNotFoundError: No module named 'lava'

In [ ]:
##custom transformation to encode images in spikes
class To_spikes(object):
    def __init__(self,time_steps, vth, img_H, img_W=None): ##return a np.array of (HxW,timesteps)
        if img_W==None:
            img_W=img_H
        self.shape = img_H*img_W
        self.vth =vth
        self.time_steps = time_steps

        self.v=np.zeros((self.shape,))

    def __call__(self, sample):
        
        out =[]
        sample = np.array(sample,dtype=np.int32)
        sample = sample -127
        sample = np.reshape(sample, (self.shape,))
        for i in range(self.time_steps):
           
            self.v = self.v + sample
            tmp = self.v > self.vth
            self.v[tmp] = 0

            if i==0:
                out=np.array([tmp])
            else:
                out=np.vstack((out,tmp))
        
        self.v[:] = 0
        
        return out.T.astype(np.float32)


class squeeze_Tensor(object):
    
    def __init__(self, dim):
        self.dim = dim

    def __call__(self, sample):
        return sample.squeeze(self.dim)

In [ ]:
transformation = transforms.Compose([To_spikes(128, 1, 28),transforms.ToTensor(),squeeze_Tensor(0) ])
testing_set  = datasets.MNIST(root='./data', train=False, download=True, transform=transformation)
testset =Subset(testing_set, range(1))
net = hdf5.Network(net_config='./Trained/network.net') 

print(net)

print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.block:5s} : {l.name:10s}, shape : {l.shape}')

In [ ]:
# Instantiate the processes
dataloader = io.dataloader.SpikeDataloader(dataset= testset) 
print("dataloader out port shape", dataloader.s_out.shape)
print("net in port shape", net.inp.shape)
output_logger = io.sink.RingBuffer(shape=net.out_layer.shape, buffer=128)
gt_logger = io.sink.RingBuffer(shape=(1,), buffer=128)

# Connect the input to the network:
dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(net.inp)

# Connect network-output to the output process
net.out_layer.out.connect(output_logger.a_in)

In [ ]:
net.run(condition=rcnd.RunSteps(128), run_cfg=rcfg.Loihi1SimCfg(select_tag='fixed_pt'))
output = output_logger.data.get()
gts = gt_logger.data.get()
net.stop()


In [ ]:
out_event = slayer.io.tensor_to_event(output.reshape((1,) + output.shape))


In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))

ax.plot(out_event.t, out_event.x, '.', markersize=2)
ax.set_title('Lava')
ax.set_xlabel('time')
print("label for current sample",testset[0][1])
#WTA Layer output

In [ ]:
output